# Tutorial 8: Manual region segmentation

This tutorial demonstrates the classification of each spot based on our manual region segmentation by labelme software, and saves the region segmentation file (label.csv). Here, We use the human bronchiolar adenoma (BA) data generated by 10x Visium. The processed data can be downloaded from https://zenodo.org/record/8137326/files/Human_Bronchiolar_Adenoma.rar?download=1.

## Preparation

In [6]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import cv2
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import os
import sys
import json
from stGCL.process import prefilter_genes,prefilter_specialgenes,set_seed,refine_nearest_labels
from stGCL import utils

In [7]:
address = "/home/dell/stproject/stGCL/Data/Human_Bronchiolar_Adenoma/spatial/tissue_hires_image.png"
image = cv2.imread(address)
f = open("/home/dell/stproject/stGCL/Data/Human_Bronchiolar_Adenoma/spatial/scalefactors_json.json", 'r')

content = f.read()
a = json.loads(content)
f.close()
scale = a["tissue_hires_scalef"]
img_new = image.copy()

In [8]:
image_loc_in    = pd.read_csv('/home/dell/stproject/stGCL/Data/Human_Bronchiolar_Adenoma/spatial/tissue_positions_list.csv', 
                              header = None, index_col = 0)
image_loc_in.columns=["isin","xarray","yarray","x","y",]
# image_loc_in["label"]="health"
image_loc_in.loc[image_loc_in["isin"]==1,"ground_truth"]="Normal"

In [9]:
sh_fname        = '/home/dell/stproject/stGCL/Data/Human_Bronchiolar_Adenoma/HBA_label.json'
color_dict={"Bronchus":255,"Tumor":0,"Blood vessel":130}
with open(sh_fname, 'r') as f:
	sh_json = json.load(f)
Height=sh_json["imageHeight"]
Width=sh_json["imageWidth"]

In [10]:
for sh in sh_json['shapes']:
    temp_celltype = sh['label']
    pts = np.squeeze(np.array(sh['points']))
    for i in pts:
        x = i[1] * scale* -1
        y = i[0] * scale* -1
        img_new[int(x - 4):int(x + 4), int(y - 4):int(y + 4), :] = color_dict[temp_celltype]
cv2.imwrite('/home/dell/stproject/stGCL/Data/Human_Bronchiolar_Adenoma/maplabel.jpg', img_new)

True

## Get label

In [11]:
for sh in sh_json['shapes']:
    temp_celltype = sh['label']
    id = sh['group_id']
    pts = np.squeeze(np.array(sh['points']))
    for barcode, imagerow, imagecol in zip(image_loc_in.index, image_loc_in["x"], image_loc_in["y"]):
        is_in = utils.judge_in(pts, (imagecol*-1+Width+0.00001, imagerow*-1+Height+0.00001))
        isin = image_loc_in.loc[barcode, "isin"]
        if is_in & isin==1:
            image_loc_in.loc[barcode, "ground_truth"]=temp_celltype
            print(barcode,temp_celltype)
image_loc_in=image_loc_in.loc[image_loc_in["isin"] == 1]
image_loc_in["ground_truth"].to_csv("/home/dell/stproject/stGCL/Data/Human_Bronchiolar_Adenoma/label.csv")

CTGACTGCGCAGCTCG-1 Bronchus
GTAATAAAGGGCTCCC-1 Bronchus
GTGAACTCCCATTCGA-1 Bronchus
GTGGTTACTTCTTTCG-1 Bronchus
TCAGAACCTCCACAGG-1 Bronchus
TCCCACTCTCTTCCGG-1 Bronchus
TCGGGAACGTGCCTAG-1 Bronchus
TAACCTACCGTCCGAG-1 Bronchus
CTTAGTAGGCCTACAG-1 Bronchus
CTAGATGTGAGTGTAA-1 Bronchus
ACTCCCGAATTCGTTT-1 Bronchus
GTTCATCGTTTGGCTG-1 Bronchus
ACTTTACCCTCATGAA-1 Bronchus
GCGAGAGTTGCGTCCA-1 Bronchus
GTTCGGGCGTACCATT-1 Bronchus
CGACTTTGTATAGCCT-1 Bronchus
TGACATCGAGCGGACC-1 Bronchus
GCGTAAATGGCCATAA-1 Bronchus
ATTGTACAACTCGGCT-1 Bronchus
TACGCTATAGAAACCT-1 Bronchus
CACCCAAATCTTATGT-1 Bronchus
AGATGATGGAGTCTGG-1 Bronchus
CCACGGTGCCCGGTAG-1 Bronchus
TCAAGAAATACTAGCT-1 Bronchus
AGGTATAATTGATAGT-1 Bronchus
ACGATCATCTTGTAAA-1 Bronchus
GAAAGAACAGCGTTAT-1 Bronchus
CTAGGTCTGAAGGAAT-1 Bronchus
ATATCAACCTACAGAG-1 Bronchus
AAATAGCTTAGACTTT-1 Bronchus
GCGACATGTAAACATC-1 Bronchus
ATAAGTAGGGCGACTC-1 Bronchus
CGCTAGAGACCGCTGC-1 Bronchus
ATAGTTCCACCCACTC-1 Bronchus
GCAGACCCAGCACGTA-1 Bronchus
TAGACTACCTAGCGTT-1 B

GGATCAGAGCCATCAG-1 Tumor
GTCCCAATCATCCCGC-1 Tumor
TGCCACACTAGAGGAA-1 Tumor
TGAATTTCACTTGCCT-1 Tumor
CTCAGATTGTGATAAG-1 Tumor
CGGACGTTACTTGAAG-1 Tumor
CCGCCTTGCGATGTCG-1 Tumor
CCAGTCAAATCTCTTA-1 Tumor
AAACAGCTTTCAGAAG-1 Tumor
TAGCTGATGTGAAGCG-1 Tumor
CTTATGCGCTCAGGGC-1 Tumor
TGGCTCTTGTCGCGTA-1 Tumor
TTGTGAGGCATGACGC-1 Tumor
CATAAGCTCTCCGTCT-1 Tumor
CCTTCTCAGCGTTCCT-1 Tumor
TGCAGCTACGTACTTC-1 Tumor
CTCTACACTGGCGATT-1 Tumor
TGCAGATCGTCCTAGG-1 Tumor
TGCCAAAGTCAGACTT-1 Tumor
CATGCCAACTCGCAAA-1 Tumor
TCGTGTCACGCTGACA-1 Tumor
CTTGCAACCGCCTCCT-1 Tumor
ATCCAATGGAGGGTCC-1 Tumor
GCAACCACGGCCGCGT-1 Tumor
CGCTTATTCCCGGTCG-1 Tumor
TTACTCTGGTACGTAC-1 Tumor
GTGGTTTCCGCCTTTC-1 Tumor
ATAGGCGGCTATAGAA-1 Tumor
GTGCCATCACACGGTG-1 Tumor
CCCAACATACGTCGCG-1 Tumor
CGGTGCAGATAGAACG-1 Tumor
GGGCGGGTTCCCTACG-1 Tumor
TGAGCCATACAGTCTC-1 Tumor
CTCCGCCCACATGAGG-1 Tumor
GTTGAACCGGTTCCAT-1 Tumor
TTGACTACCATATGGT-1 Tumor
ACCATCGTATATGGTA-1 Tumor
TGCGTAAGAACCTGAT-1 Tumor
AGAAGGTTGCCGAATT-1 Tumor
AGGACGACCCATTAGA-1 Tumor


TCAAAGTCACGGCGTC-1 Tumor
TGGGTTTCGGGCGTAC-1 Tumor
TGCTATGGCAAAGGGA-1 Tumor
TTCACGGTCGTCACGT-1 Tumor
CGTAAAGCAAGAAATC-1 Tumor
CTGATCCCTTTATGCA-1 Tumor
GAGTATACCCTAATCA-1 Tumor
CCACGGCAGGTGTAGG-1 Tumor
ACGTACTTTGGCACGG-1 Tumor
CCTGAGAATAAATGCA-1 Tumor
TTGATGCCGCTCGTCG-1 Tumor
GCCAGGAAAGAACACT-1 Tumor
TCCGACCGCTAATCAA-1 Tumor
TATATCAAAGTGATCT-1 Tumor
CCTCGGTTTCCTTGCC-1 Tumor
TAAATTTAGTAACACC-1 Tumor
CTCGTTACGGCTACCA-1 Tumor
GCCGTCGGTTTCGGGC-1 Tumor
GTTCCGTCCGCCTGCA-1 Tumor
CTAGGGATAGGGACAA-1 Tumor
GTCCAGGCACGTGTGC-1 Tumor
GAGGAATATCTCTTTG-1 Tumor
CGTTCAAGGAAACGGA-1 Tumor
GTGGGAACAAACCGGG-1 Tumor
GGGCACGTAGTACTGT-1 Tumor
ACTACAAAGAGAGGTG-1 Tumor
GGTGTAGGTAAGTAAA-1 Tumor
CCTGTTCAACCTCGGG-1 Tumor
TGAGGTGTGTGGCGGA-1 Tumor
ATTGTCTGTTTCATGT-1 Tumor
GCCGGTCGTATCTCTC-1 Tumor
ATATGGGATAGCAACT-1 Tumor
GATAGAACCCGCTAGG-1 Tumor
ATCCCATTTCCGTGGG-1 Tumor
GATGCTACCTTCGATG-1 Tumor
TGCAAAGTTCGTCTGT-1 Tumor
GTTCAGTCGCCAAATG-1 Tumor
GTCTCCGCCTCAATAC-1 Tumor
AGTAGAAGGCGCCTCA-1 Tumor
GTTGTAGATTTATGAG-1 Tumor
